In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import astropy
import astropy.units as u
import scipy
import scipy.constants as cnst
from scipy.signal import savgol_filter 
import glob
import re
# import util

# Constants
R_sol = 8.23e3 * u.parsec

# pd output style
pd.set_option('display.float_format','{:.9e}'.format)

In [69]:
def freq2vel(freq, units = False):
    freq_0 = 1.420405751768e9
    # vel = (freq_0-freq)/freq_0*cnst.c/1000
    vel = (1-(freq/freq_0)**2)/(1+(freq/freq_0)**2)*cnst.c/1000
    if units:
        return vel*u.kilometer/u.second
    else:
        return vel

def power2temp(power, units = False):
    if units:
        return power*90*u.K
    else:
        return power*90

def volt2power(volt, units = False):
    if units:
        return np.power(volt,2)/50*u.power
    else:
        return np.power(volt,2)/50
    
def volt2temp(volt, units = False):
    if units:
        return power2temp(volt2power(volt), units = True)
    else:
        return power2temp(volt2power(volt))

def read_dat(file_path):
    df = pd.read_csv(file_path, sep = '\t', skiprows = 1, names = ['Frequency', 'Amplitude'])
    return df

def smooth(arr):
    return scipy.ndimage.gaussian_filter1d(arr,5)    

def find_bg_signal(df, graph=False):
    if graph:
        sns.histplot(data =df, x= 'Amplitude', bins = 100, kde= True)
    hist, bin_edges = np.histogram(df['Amplitude'], bins=200, density = False)
    eval_points = np.linspace(np.min(bin_edges), np.max(bin_edges), num = 200)
    kde_max = eval_points[np.argmax(scipy.stats.gaussian_kde(df['Amplitude']).pdf(eval_points))]
    # hist_max = (bin_edges[np.argmax(hist)] + bin_edges[np.argmax(hist)+1] )/2
    return kde_max

def remove_width(arr, peaks, width):
    n = arr.size
    for peak in peaks:
        arr[peak] = np.nan
        for i in range(1,(width-1)//2):
            if peak - i > 0:
                arr[peak-i] = np.nan
            if peak + i < n:
                arr[peak+i] = np.nan
    return pd.Series(arr).interpolate(method='linear', limit_direction='both').to_numpy()

def remove_spike(df, smoothing = False):
    bg = find_bg_signal(df)
    std = np.std(df['Amplitude'])
    noise_std = np.std(df['Amplitude'][df['Amplitude']<bg])
    amp = df['Amplitude'].to_numpy()
    peaks = scipy.signal.find_peaks_cwt(amp, np.arange(1,7), min_snr=2, min_length=1)
    
    unpeaked = df['Amplitude'].to_numpy()
    new_unpeaked = remove_width(unpeaked, peaks, 13)
    
    if smoothing == True:
        new_unpeaked = smooth(new_unpeaked)
    df['Amplitude'] = new_unpeaked
    
    bg = find_bg_signal(df)
    # vel = freq2vel(df['Frequency'])
    
    # plt.plot(vel,new_unpeaked-bg)
    # xmin = vel.to_numpy()[0]
    # xmax = vel.to_numpy()[-1]
    # plt.hlines(y=0, xmin = xmin, xmax = xmax, linewidth=1, color='r')
    
    # plt.hlines(y=bg-noise_std, xmin = xmin, xmax = xmax, linewidth=1, color='r')
    
    return bg, new_unpeaked

def cal_rot(v_rec, disp, v_earth, theta):
    v_rot = v_rec - disp + v_earth * np.sin(theta) * u.km / u.s
    r_min = np.sin(theta) * R_sol
    return r_min, v_rot

def vel_correction(vel):
    
    return


    

In [70]:
df = read_dat('galaxy\galaxy_lon_20.dat')
# print(df)
bg, unpeaked = remove_spike(df)
# df['Amplitude'] = new_unpeaked
df

,Frequency,Amplitude
0,1.419400000e+09,1.699915000e-05
1,1.419403333e+09,1.700055136e-05
2,1.419406667e+09,1.700195273e-05
3,1.419410000e+09,1.700335409e-05
4,1.419413333e+09,1.700475545e-05
...,...,...
596,1.421386667e+09,1.666690000e-05
597,1.421390000e+09,1.666690000e-05
598,1.421393333e+09,1.666690000e-05
599,1.421396667e+09,1.666690000e-05


In [75]:
paths = glob.glob("galaxy/galaxy_lon_??.dat")
print(paths)
for path in paths:
    df = read_dat(path)
    bg, unpeaked = remove_spike(df)
    print('galaxy/unpeaked/'+path[7:-4]+'_unpeaked.dat')
    df.to_csv('galaxy/unpeaked/'+path[7:-4]+'_unpeaked.dat', sep = "\t", float_format = '{:.9e}'.format, index = False, header = ['Frequency, Hz', 'Amplitude, Volts'])
    
    

['galaxy\\galaxy_lon_00.dat', 'galaxy\\galaxy_lon_05.dat', 'galaxy\\galaxy_lon_10.dat', 'galaxy\\galaxy_lon_15.dat', 'galaxy\\galaxy_lon_20.dat', 'galaxy\\galaxy_lon_25.dat', 'galaxy\\galaxy_lon_30.dat', 'galaxy\\galaxy_lon_35.dat', 'galaxy\\galaxy_lon_40.dat', 'galaxy\\galaxy_lon_45.dat', 'galaxy\\galaxy_lon_50.dat', 'galaxy\\galaxy_lon_55.dat', 'galaxy\\galaxy_lon_60.dat', 'galaxy\\galaxy_lon_65.dat', 'galaxy\\galaxy_lon_70.dat', 'galaxy\\galaxy_lon_75.dat', 'galaxy\\galaxy_lon_80.dat', 'galaxy\\galaxy_lon_85.dat', 'galaxy\\galaxy_lon_90.dat']
galaxy/unpeaked/galaxy_lon_00_unpeaked.dat
galaxy/unpeaked/galaxy_lon_05_unpeaked.dat
galaxy/unpeaked/galaxy_lon_10_unpeaked.dat
galaxy/unpeaked/galaxy_lon_15_unpeaked.dat
galaxy/unpeaked/galaxy_lon_20_unpeaked.dat
galaxy/unpeaked/galaxy_lon_25_unpeaked.dat
galaxy/unpeaked/galaxy_lon_30_unpeaked.dat
galaxy/unpeaked/galaxy_lon_35_unpeaked.dat
galaxy/unpeaked/galaxy_lon_40_unpeaked.dat
galaxy/unpeaked/galaxy_lon_45_unpeaked.dat
galaxy/unpeaked/ga